# Visualize multiple Robots

For the visualization of multiple robots, there are some extra steps necessary. In this tutorial, we are going to build two modular robots from IMPROV modules and animate them in the same window.

Before you start, make sure to do the first animation tutorial!

In [2]:
import numpy as np
import meshcat.animation

import timor
from timor import ModuleAssembly, ModulesDB, Transformation
from timor.utilities.file_locations import get_module_db_files
from timor.utilities.visualization import animation, MeshcatVisualizerWithAnimation

As always, **loading the module set** has to come first. Next, we are going to build two serial robots based on the modules available.

In [3]:
improv_modules = ModulesDB.from_json_file(get_module_db_files('IMPROV'))
assembly_one = ModuleAssembly.from_serial_modules(improv_modules, ('1', '21', '4', '21', '23', '12'))
assembly_two = ModuleAssembly.from_serial_modules(improv_modules, ('1', '21', '4', '21', '5', '23', '7', '12'))

2024-10-23 14:19:58,253 Timor WARNING No collision model given for body 1
2024-10-23 14:19:58,254 Timor WARNING EmptyGeometry does not use parameters, but they were given.


Curious what ID belongs to which robot? Enter `improv_modules.debug_visualization()` to get an idea about the module set. It will look like this:


![debug_visualization](img/IMPROV_modules.png)

On the control panel on the right side, you can hide/unhide coordinate systems or modules and get an idea about them before deciding for an assembly.

Next, we have to transform the assemblies to pinocchio models - also, we want to have them at different locations in the world, so we change the placement of the base connector reference frame for the larger robot by moving it 1m in x-direction:

In [4]:
small_robot = assembly_one.to_pin_robot()
large_robot = assembly_two.to_pin_robot()
large_robot.set_base_placement(Transformation.from_translation([1, 0, 0]))

The visualizer will display the robots in the control panel under their name - as an assembly does not come with a given name (as e.g. a URDF model would), we assign them manually:

In [5]:
small_robot._name = 'small'
large_robot._name = 'large'

Next, we generate a motion trajectory to perform - these are just some arbitrary values - feel free to play around with them:

In [6]:
q0 = np.zeros((6,))
q1 = np.array([np.pi/4, -np.pi/3, -np.pi/2, 0, np.pi/3, np.pi/2])
q2 = -q1

move_right = np.linspace(q0, q1, 100)
move_left = np.linspace(q1, q2, 200)
move_back = np.linspace(q2, q0, 100)
stand_still = np.vstack([np.expand_dims(q0, 0)] * 25)
trajectory = np.vstack([move_right, move_left, move_back, stand_still])

The default `animation` method provided with timor is designed for one single robot - so we have to directly interact with the meshcat visualizer provided with pinochio. What we do is:

1. Set up a visualizer with animation capabilities and load the small robot there.
2. Generate a second visualizer for the large robot and make it refer to the same viewer object s.t. both robots will be laoded in the same window.
3. Create an animation object
4. For every step in our trajectory, generate one frame for the animation and use the visualizer method `create_animation_frame` to update the animation
5. Load the animation in the viewer window

In [7]:
v = MeshcatVisualizerWithAnimation.from_MeshcatVisualizer(small_robot.visualize())
new_viz = MeshcatVisualizerWithAnimation()
new_viz.viewer = v.viewer
large_robot.visualize(visualizer=new_viz)
anim = meshcat.animation.Animation(default_framerate=60)

for i in range(0, trajectory.shape[0]):
    with anim.at_frame(v.viewer, i) as frame:
        small_robot.update_configuration(trajectory[i, :])
        large_robot.update_configuration(trajectory[i, :])
        v.create_animation_frame(timor.visualization.VISUAL, frame)
        new_viz.create_animation_frame(timor.visualization.VISUAL, frame)

v.viewer.set_animation(anim, repetitions=2)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


AttributeError: 'MeshcatVisualizerWithAnimation' object has no attribute 'static_objects'

Congratulations - that's it! The same process works without setting an animation, of course. Just stop the tutorial at the point where we start building the animation and you will get a static visualization.

**Tip**: You can also do in-notebook visualization instead of the browser-based animation:

In [8]:
v.viewer.jupyter_cell(height=800)